In [8]:
cd Desktop

[WinError 2] The system cannot find the file specified: 'Desktop'
C:\Users\GC Lab\Desktop


In [9]:
#import dataset

import pandas as pd
data=pd.read_json("News_Category_Dataset.json",lines=True)
data.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [10]:
# merge headline column and short_description to get one extra feature named important feature

data['imp'] = data['short_description'].astype(str) + data['headline']

In [11]:
#import required library 

import pandas as pd
import numpy as np
pd.options.display.max_columns = 200
pd.options.mode.chained_assignment = None

from nltk.tokenize import word_tokenize, sent_tokenize
#from nltk.corpus import stopwords
#stop = set(stopwords.words('english'))
from string import punctuation

from collections import Counter
import re
import numpy as np

from tqdm import tqdm_notebook
tqdm_notebook().pandas()

In [12]:
# shape of dataset

data.shape

(124988, 7)

In [13]:
# remove duplicates and null value from imp column

data = data.drop_duplicates('imp')
data = data[~data['imp'].isnull()]
print(data.shape)

(124765, 7)


In [14]:
# these are most frequent stop words here used my own generated stopwords rather than predefied NLP having very less no. of stop words approx 150
# stopwords.txt is imported drom my local system rather than python pre-defined 

stop_words = []
f = open('stopwords.txt', 'r')
for l in f.readlines():
    stop_words.append(l.replace('\n', ''))
additional_stop_words = ['t', 'will']
stop_words += additional_stop_words

print(len(stop_words))

668


In [15]:
# these are stopwords and punctuation which is useless in our corpus so we should remove this

print(list(stop_words))
print(list(punctuation))

['a', 'able', 'about', 'above', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'added', 'adj', 'affected', 'affecting', 'affects', 'after', 'afterwards', 'again', 'against', 'ah', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'announce', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apparently', 'approximately', 'are', 'aren', 'arent', 'arise', 'around', 'as', 'aside', 'ask', 'asking', 'at', 'auth', 'available', 'away', 'awfully', 'b', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'begin', 'beginning', 'beginnings', 'begins', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'between', 'beyond', 'biol', 'both', 'brief', 'briefly', 'but', 'by', 'c', 'ca', 'came', 'can', 'cannot', "can't", 'cause', 'causes', 'certain', 'certainly', 'co', 'com', 'come', 'comes', 'cont

In [16]:
# clean the imp column of  text by changing into simple words free from non-ascii converted into lower case 

from functools import reduce
def _removeNonAscii(s): 
    return "".join(i for i in s if ord(i)<128)


def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = text.replace('(ap)', '')
    text = re.sub(r"\'s", " is ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    text = re.sub('[^a-zA-Z ?!]+', '', text)
    text = _removeNonAscii(text)
    text = text.strip()
    return text

In [17]:
# change the whole lines into tockens

def tokenizer(text):
    text = clean_text(text)    
    tokens = [word_tokenize(sent) for sent in sent_tokenize(text)]
    tokens = (reduce(lambda x,y: x+y, tokens,[]))
    tockens = tokens[:] 
    filtered_sentence = [w for w in tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
     
    
    
    tokens = list(filter(lambda token: token not in (list(stop_words) + list(punctuation)) , tokens))
    return filtered_sentence


In [18]:
#tockanize words of short_description free from stopwords punctuation 

data['tokens'] = data['imp'].progress_map(lambda d: tokenizer(d))

In [19]:
# import feature column ate converted into tockens free from stopwords 

for descripition, tokens, category in zip(data['imp'].head(5), data['tokens'].head(5),data['category'].head(5)):
    print('\nimportant feature: ', descripition)
    print('\ntokens: ', tokens)
    print('\ncategory: ',category)
    


important feature:  She left her husband. He killed their children. Just another day in America.There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV

tokens:  ['left', 'husband', 'killed', 'children', 'day', 'america', 'mass', 'shootings', 'texas', 'week', 'tv']

category:  CRIME

important feature:  The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.Hugh Grant Marries For The First Time At Age 57

tokens:  ['actor', 'longtime', 'girlfriend', 'anna', 'eberstein', 'tied', 'knot', 'civil', 'ceremony', 'hugh', 'grant', 'marries', 'time', 'age']

category:  ENTERTAINMENT

important feature:  The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork

tokens:  ['actor', 'dems', 'ass', 'kicking', 'fighting', 'hard', 'donald', 'trump', 'jim', 'carrey', 'blasts', 'castrato', 'adam', 'schiff', 'democrats', 'artwork']

category:  ENTERTAINMENT

importa

In [20]:
# data2 contains less no. of tuples then in original dataset

data2 = data.sample(80000, random_state=42)
data2.reset_index(inplace=True, drop=True)
data2.head(2)

,authors,category,date,headline,link,short_description,imp,tokens
0,Zach Carter,POLITICS,2016-07-23,Hillary Clinton Does Not Understand Her Own Su...,https://www.huffingtonpost.com/entry/hillary-c...,She just pointlessly poked them in the eye.,She just pointlessly poked them in the eye.Hil...,"[pointlessly, poked, eye, hillary, clinton, un..."
1,"Peter Greenberg, ContributorTravel Editor for ...",TRAVEL,2014-05-02,"Incredible Seaplane Views of Anchorage, Alaska",https://www.huffingtonpost.com/entry/hidden-ge...,"Head over to Lake Hood, the busiest seaplane p...","Head over to Lake Hood, the busiest seaplane p...","[head, lake, hood, busiest, seaplane, port, ai..."


In [21]:
# tf-idf vectorization performed on tockenized dataset 

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5, analyzer='word', ngram_range=(1,2), stop_words=stop_words)
vx = vectorizer.fit_transform(list(data['tokens'].map(lambda tokens: ' '.join(tokens))))


In [22]:
# shape of tf-idf sparse matrix

vx.shape

(124765, 43817)

In [23]:
# sparse matrix changed into array

dense = vx.toarray()


In [24]:
# datasets is splitted into train and test 

from sklearn.model_selection import train_test_split
des_tr, des_te, cat_tr, cat_te = train_test_split(dense,data['category'],test_size=0.2)

In [25]:
# dataset is fit on linear support vector

from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(des_tr,cat_tr)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [26]:
# prediction is made on test data and accuracy is calculate

y_pred = model.predict(des_te)
acc = accuracy_score(cat_te, y_pred)
print("Accuracy {:.2f}".format(acc*100))

Accuracy 61.57


In [27]:
# these are 50 sample of actual value and predicted result on test data we can change range to get more sample
# these results shows model doesn't predicted weird result even if wrong prediction is somewhat similar to actual 

p=np.asarray(cat_te)
for x in range(5000,5050):
    print("actual:" ,p[x] , "           predicted:" ,y_pred[x])
   

actual: SCIENCE            predicted: SCIENCE
actual: POLITICS            predicted: POLITICS
actual: IMPACT            predicted: RELIGION
actual: BLACK VOICES            predicted: WOMEN
actual: SCIENCE            predicted: GREEN
actual: HEALTHY LIVING            predicted: HEALTHY LIVING
actual: IMPACT            predicted: SCIENCE
actual: PARENTS            predicted: PARENTS
actual: POLITICS            predicted: POLITICS
actual: POLITICS            predicted: POLITICS
actual: COMEDY            predicted: COMEDY
actual: ENTERTAINMENT            predicted: ENTERTAINMENT
actual: PARENTS            predicted: PARENTS
actual: POLITICS            predicted: GREEN
actual: POLITICS            predicted: POLITICS
actual: GOOD NEWS            predicted: SPORTS
actual: COMEDY            predicted: POLITICS
actual: IMPACT            predicted: EDUCATION
actual: POLITICS            predicted: POLITICS
actual: PARENTS            predicted: EDUCATION
actual: PARENTS            predicted: PAREN

In [28]:
# linear regression model is trained on the dataset and accuracy is calculated

from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(des_tr,cat_tr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
y_pred = model.predict(des_te)
acc = accuracy_score(cat_te, y_pred)
print("Accuracy {:.2f}".format(acc*100))

Accuracy 59.89


In [30]:
# multinominal naive baise model is fit and prediction is measured  

from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(des_tr,cat_tr)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [31]:
y_pred = model.predict(des_te)
acc = accuracy_score(cat_te, y_pred)
print("Accuracy {:.2f}".format(acc*100))

Accuracy 45.57
